This notebook is unstable scratch paper for my ongoing experiments.

In [ ]:
#Setup

import sys
sys.path.append('./src')

conv = True
preprocess = True
#env_id = "ALE/Pong-v5"
env_id = "PongNoFrameskip-v4"
fbfile = "pong/pong_serious.dat"
#fbfile = "pong_synth.dat"
#env_id = 'CartPole-v1'
#fbfile = 'testcartpole.dat'
#env_id = 'LunarLander-v2'
#fbfile = "lunarlandersynth.dat"
#fbfile = "lunarlandernormal.dat"
#fbfile = "lunarlander2.dat"
#env_id = "BipedalWalker-v3"
#fbfile = "bipedal1.dat"

save_dir = "pong"

from training_ui import TrainingWindow

tw = TrainingWindow(env_id, fbfile, save_dir, conv, preprocess)

In [ ]:
#fb.viewComparisons()